<a href="https://colab.research.google.com/github/peculab/AI4JUBO/blob/main/JuboDeath_V8_pureData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#訓練資料是 mortality_2020_2023_0910/training_data_v2_0917
#外部驗證資料是 mortality_2024_0910/external_validation_0917

#次族群

- <= 85 & > 85
- ADL 變好 & ADL 變差
- 男性 & 女性

In [ ]:
!pip install shap plotly xgboost --quiet

In [ ]:
!pip uninstall shap -y
!pip install shap --no-deps

Found existing installation: shap 0.48.0
Uninstalling shap-0.48.0:
  Successfully uninstalled shap-0.48.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.1 MB/s eta 0:00:00


In [ ]:
!pip install ace_tools

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, accuracy_score
from IPython.display import display
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    classification_report, confusion_matrix, mean_absolute_error, r2_score
)

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

外部資料讀入

In [ ]:
# read data and put it in a dataframe
# 在 google 工作表載入外部資料 gsheets

gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1jXuepVDDrnhOKYMqy-RQKeQlWr7atvKdeq_Y0PosMDM/edit?usp=sharing')
worksheet = gsheets.worksheet("external_validation_0917")  # 指定分頁名稱

worksheet = worksheet.get_all_records()
external = pd.DataFrame(worksheet)
external = external.apply(lambda col: pd.to_numeric(col.astype(str).str.replace(',', '').str.strip(), errors='coerce'))
external.head()

,H01_NUM,dbname,入家日期,結案日期,死亡標記,觀察天數,性別_is_male,預估年齡,DNR_flag,ADL_總分_max,...,first_ 意識總分,last_ 意識總分,意識總分Max,意識總分_diff,意識分級,使用呼吸輔具,first_has_feeding_tube,last_has_feeding_tube,diff_has_feeding_tube,had_fall
0,1376,NaN,NaN,NaN,0,197,1,77,0,90,...,15.0,15.0,15.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0
1,1322,NaN,NaN,NaN,0,327,1,92,0,10,...,12.0,12.0,12.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0
2,1319,NaN,NaN,NaN,0,255,1,78,1,5,...,11.0,10.0,11.0,-1.0,NaN,0.0,0.0,0.0,0.0,0.0
3,1333,NaN,NaN,NaN,0,293,1,82,1,10,...,12.0,12.0,12.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0
4,1452,NaN,NaN,NaN,0,341,0,80,1,10,...,11.0,11.0,11.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0


In [ ]:
external.describe(include='all').T

,count,mean,std,min,25%,50%,75%,max
H01_NUM,6339.0,1383.704685,767.419894,4.0,1184.0,1265.0,1386.000000,20295.000000
dbname,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
入家日期,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
結案日期,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
死亡標記,6339.0,0.280959,0.449503,0.0,0.0,0.0,1.000000,1.000000
觀察天數,6339.0,210.107588,109.964750,0.0,127.5,235.0,298.000000,365.000000
性別_is_male,6339.0,0.508440,0.499968,0.0,0.0,1.0,1.000000,1.000000
預估年齡,6339.0,78.607825,11.742108,1.0,72.0,81.0,87.000000,124.000000
DNR_flag,6339.0,0.456854,0.498174,0.0,0.0,0.0,1.000000,1.000000
ADL_總分_max,6339.0,29.695536,31.687105,0.0,0.0,20.0,50.000000,100.000000


In [ ]:
ex_missing_info = external.isnull().sum().to_frame(name='Missing Count')
ex_missing_info['Missing Ratio'] = (ex_missing_info['Missing Count'] / len(external)).round(4)
ex_missing_info = ex_missing_info.sort_values(by='Missing Ratio', ascending=True)
ex_missing_info

,Missing Count,Missing Ratio
H01_NUM,0,0.0000
性別_is_male,0,0.0000
觀察天數,0,0.0000
死亡標記,0,0.0000
預估年齡,0,0.0000
ADL_Max,0,0.0000
ADL_總分_max,0,0.0000
DNR_flag,0,0.0000
ADL_Min,0,0.0000
ADL_last_CouldNot,0,0.0000


訓練資料讀入

In [ ]:
# read data and put it in a dataframe
# 在 google 工作表載入訓練資料 gsheets

gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1_D_ryADEhFBC0322zwlI7uQZz5xELWlZ-3CO549cLS0/edit?usp=sharing')
worksheet = gsheets.worksheet("training_data_v2_0917")  # 指定分頁名稱

worksheet = worksheet.get_all_records()
df = pd.DataFrame(worksheet)
df = df.apply(lambda col: pd.to_numeric(col.astype(str).str.replace(',', '').str.strip(), errors='coerce'))
df.head()

,H01_NUM,dbname,入家日期,結案日期,死亡標記,觀察天數,性別_is_male,預估年齡,DNR_flag,ADL_總分_max,...,first_ 意識總分,last_ 意識總分,意識總分Max,意識總分_diff,意識分級,使用呼吸輔具,first_has_feeding_tube,last_has_feeding_tube,diff_has_feeding_tube,had_fall
0,1325,NaN,NaN,NaN,0,739,1,66,1,0,...,13.0,12.0,13.0,-1.0,NaN,0.0,0.0,0.0,0.0,0.0
1,1160,NaN,NaN,NaN,0,788,1,89,1,95,...,15.0,13.0,15.0,-2.0,NaN,0.0,0.0,0.0,0.0,0.0
2,1253,NaN,NaN,NaN,0,1292,0,89,0,60,...,15.0,15.0,15.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0
3,1342,NaN,NaN,NaN,0,584,0,93,1,30,...,14.0,14.0,14.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0
4,1343,NaN,NaN,NaN,0,583,1,90,0,90,...,15.0,15.0,15.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0


In [ ]:
df.describe(include='all').T

,count,mean,std,min,25%,50%,75%,max
H01_NUM,27037.0,1500.801457,4160.182209,1.0,1159.0,1220.000000,1317.000000,100463.000000
dbname,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
入家日期,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
結案日期,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
死亡標記,27037.0,0.194992,0.396202,0.0,0.0,0.000000,0.000000,1.000000
觀察天數,27037.0,634.100529,437.553151,0.0,248.0,597.000000,1014.000000,1460.000000
性別_is_male,27037.0,0.485557,0.499801,0.0,0.0,0.000000,1.000000,1.000000
預估年齡,27037.0,79.388801,11.819412,0.0,72.0,82.000000,88.000000,125.000000
DNR_flag,27037.0,0.408810,0.491623,0.0,0.0,0.000000,1.000000,1.000000
ADL_總分_max,27037.0,31.974886,33.277403,0.0,0.0,20.000000,55.000000,100.000000


In [ ]:
df_missing_info = df.isnull().sum().to_frame(name='Missing Count')
df_missing_info['Missing Ratio'] = (df_missing_info['Missing Count'] / len(df)).round(4)
df_missing_info = df_missing_info.sort_values(by='Missing Ratio', ascending=True)
df_missing_info

,Missing Count,Missing Ratio
H01_NUM,0,0.0000
性別_is_male,0,0.0000
觀察天數,0,0.0000
死亡標記,0,0.0000
預估年齡,0,0.0000
ADL_Max,0,0.0000
ADL_總分_max,0,0.0000
DNR_flag,0,0.0000
ADL_Min,0,0.0000
ADL_last_CouldNot,0,0.0000


In [ ]:
features = df_missing_info[df_missing_info['Missing Ratio']<0.3].index.tolist()

In [ ]:
features

['H01_NUM',
 '性別_is_male',
 '觀察天數',
 '死亡標記',
 '預估年齡',
 'ADL_Max',
 'ADL_總分_max',
 'DNR_flag',
 'ADL_Min',
 'ADL_last_CouldNot',
 'ADL_first_CouldNot',
 'ADL_明顯惡化',
 '六個月內住院次數',
 'ADL_first_score',
 'ADL_last_score',
 'ADL_diff_seq',
 'diff_has_denture',
 'last_has_denture',
 'first_has_denture',
 'diff_has_feeding_tube',
 'had_fall',
 'first_has_feeding_tube',
 'last_has_feeding_tube',
 '意識總分Max',
 '使用呼吸輔具',
 'ADL_std']

In [ ]:
dfNew = df[features]

In [ ]:
dfNew = dfNew.fillna(0)

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin

class HybridXGBRF(BaseEstimator, ClassifierMixin):
    def __init__(self, xgb_model=None, rf_model=None, alpha=0.5):
        self.xgb_model = xgb_model
        self.rf_model = rf_model
        self.alpha = alpha
        self._init_models()

    def _init_models(self):
        # Best Parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 800, 'subsample': 1.0}
        # "XGBClassifier": XGBClassifier(n_estimators=200, learning_rate=0.01, max_depth=5, random_state=42, eval_metric='logloss'),

        self.xgb = self.xgb_model or XGBClassifier(
            eval_metric="logloss",
            random_state=42,
            colsample_bytree=0.8,     # ✅ 降低每棵樹看到的特徵比例 → 提高多樣性
            learning_rate=0.01,       # ✅ 稍微提升學習率搭配更早停止
            max_depth=5,              # ✅ 降低單棵樹複雜度 → 降低過擬合
            n_estimators=200,         # ✅ 總樹數可略減以免累積錯誤
            subsample=1.0,            # ✅ 樣本隨機抽樣 → 提升隨機性
            verbosity=0,
            use_label_encoder=False
        )
        self.rf = self.rf_model or RandomForestClassifier(
            n_estimators=100,
            random_state=42
        )

    def fit(self, X, y):
        self._init_models()  # 每次 fit 要重設模型
        self.xgb.fit(X, y)
        self.rf.fit(X, y)
        return self

    def predict_proba(self, X):
        xgb_prob = self.xgb.predict_proba(X)[:, 1]
        rf_prob = self.rf.predict_proba(X)[:, 1]
        blended = self.alpha * xgb_prob + (1 - self.alpha) * rf_prob
        return np.vstack([1 - blended, blended]).T

    def predict(self, X):
        return (self.predict_proba(X)[:, 1] > 0.5).astype(int)

    def get_params(self, deep=True):
        return {
            'xgb_model': self.xgb_model,
            'rf_model': self.rf_model,
            'alpha': self.alpha
        }

    def set_params(self, **params):
        for param, value in params.items():
            setattr(self, param, value)
        self._init_models()  # 重新初始化模型
        return self

In [ ]:
# 要移除的欄位，是代表身分標記，以及天數
drop_columns = ['H01_NUM', '觀察天數']

# 丟掉這些欄位
dfNew = dfNew.drop(columns=drop_columns)

In [ ]:
dfNew

,性別_is_male,死亡標記,預估年齡,ADL_Max,ADL_總分_max,DNR_flag,ADL_Min,ADL_last_CouldNot,ADL_first_CouldNot,ADL_明顯惡化,...,diff_has_denture,last_has_denture,first_has_denture,diff_has_feeding_tube,had_fall,first_has_feeding_tube,last_has_feeding_tube,意識總分Max,使用呼吸輔具,ADL_std
0,1,0,66,0,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,0.0,0.000000
1,1,0,89,95,95,1,10,0,0,1,...,-1.0,0.0,1.0,0.0,0.0,0.0,0.0,15.0,0.0,49.074773
2,0,0,89,60,60,0,30,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,16.431677
3,0,0,93,30,30,1,10,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0,10.350983
4,1,0,90,90,90,0,5,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,45.434411
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27032,1,0,48,100,100,0,100,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.000000
27033,1,0,41,100,100,0,100,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.000000
27034,0,0,60,100,100,0,100,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.000000
27035,0,0,54,100,100,0,100,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.000000


In [ ]:
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import (
    confusion_matrix, roc_curve, auc, classification_report
)
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
# === 資料準備 ===
X = dfNew.drop(columns=['死亡標記'])
y = df['死亡標記']

In [ ]:
X_missing_info = X.isnull().sum().to_frame(name='Missing Count')
X_missing_info['Missing Ratio'] = (X_missing_info['Missing Count'] / len(X)).round(4)
X_missing_info = X_missing_info.sort_values(by='Missing Ratio', ascending=True)
X_missing_info

,Missing Count,Missing Ratio
性別_is_male,0,0.0
預估年齡,0,0.0
ADL_Max,0,0.0
ADL_總分_max,0,0.0
DNR_flag,0,0.0
ADL_Min,0,0.0
ADL_last_CouldNot,0,0.0
ADL_first_CouldNot,0,0.0
ADL_明顯惡化,0,0.0
六個月內住院次數,0,0.0


# 開始進行訓練

In [ ]:
X.describe(include='all').T

,count,mean,std,min,25%,50%,75%,max
性別_is_male,27037.0,0.485557,0.499801,0.0,0.0,0.0,1.000000,1.000000
預估年齡,27037.0,79.388801,11.819412,0.0,72.0,82.0,88.000000,125.000000
ADL_Max,27037.0,31.974886,33.277403,0.0,0.0,20.0,55.000000,100.000000
ADL_總分_max,27037.0,31.974886,33.277403,0.0,0.0,20.0,55.000000,100.000000
DNR_flag,27037.0,0.408810,0.491623,0.0,0.0,0.0,1.000000,1.000000
ADL_Min,27037.0,20.445131,28.644584,0.0,0.0,5.0,30.000000,100.000000
ADL_last_CouldNot,27037.0,0.019492,0.138248,0.0,0.0,0.0,0.000000,1.000000
ADL_first_CouldNot,27037.0,0.017384,0.130698,0.0,0.0,0.0,0.000000,1.000000
ADL_明顯惡化,27037.0,0.194955,0.396173,0.0,0.0,0.0,0.000000,1.000000
六個月內住院次數,27037.0,2.779672,10.889087,0.0,0.0,1.0,3.000000,618.000000


In [ ]:
!pip install lifelines

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.3/117.3 kB 13.1 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4030 sha256=32ab869246d05a1f7410b334c8f3d87305864daae9f65cdc0fccb3ea943758a6
  Stored in directory: /root/.cache/pip/wheels/50/37/21/0a719b9d89c635e89ff24bd93b862882ad675279552013b2fb
Successfully built autograd-gamma


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lifelines import CoxPHFitter
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegressionCV

# 假設 HybridXGBRF 已定義
all_models = {
    #"HybridXGBRF (Our Approach)": HybridXGBRF(alpha=1),
    "HybridXGBRF (Our Approach)": XGBClassifier(n_estimators=200, learning_rate=0.01, max_depth=5, random_state=42, eval_metric='logloss', subsample=1.0, verbosity=0),
    "LogisticRegression (max_iter=200)": LogisticRegression(max_iter=200),
    "XGBClassifier": XGBClassifier(n_estimators=500, learning_rate=0.01, max_depth=3, random_state=42, eval_metric='logloss'),
    "RandomForestClassifier": RandomForestClassifier(n_estimators=100, random_state=42),
    "LogisticRegression (max_iter=1000)": LogisticRegression(max_iter=1000),

    # 🔽 新增未測試模型
    "Ridge": make_pipeline(StandardScaler(), LogisticRegression(penalty='l2', solver='saga', max_iter=1000, random_state=42)),
    "Lasso": make_pipeline(StandardScaler(), LogisticRegression(penalty='l1', solver='saga', max_iter=1000, random_state=42)),
    "Elastic": make_pipeline(StandardScaler(), LogisticRegression(penalty='elasticnet', solver='saga', l1_ratio=0.5, max_iter=1000, random_state=42)),
}

In [ ]:
import copy
from sklearn.base import clone
from sklearn.impute import SimpleImputer
from sklearn.exceptions import NotFittedError
from sklearn.utils.validation import check_is_fitted

# Prepare CV and ROC Figure
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
fig_roc = go.Figure()
mean_fpr = np.linspace(0, 1, 100)

results = []
trained_models = {}

for model_name, model in all_models.items():
    print(f"▶ Running CV for: {model_name}")
    accs, precs, recalls, f1s, aucs = [], [], [], [], []
    all_cm = np.zeros((2, 2), dtype=int)
    tprs = []

    for fold, (train_idx, test_idx) in enumerate(skf.split(X, y)):
        X_train, X_test = X.iloc[train_idx].copy(), X.iloc[test_idx].copy()
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        model_fold = clone(model)

        try:
            # 嘗試直接 fit（若模型支援 NaN，會成功）
            model_fold.fit(X_train, y_train)

        except ValueError as e:
            if "Input X contains NaN" in str(e):
                print(f"⚠️ Missing value detected for {model_name} (fold {fold+1}) — applying median imputation.")

                imputer = SimpleImputer(strategy='median')
                X_train = pd.DataFrame(imputer.fit_transform(X_train), columns=X.columns)
                X_test = pd.DataFrame(imputer.transform(X_test), columns=X.columns)

                model_fold.fit(X_train, y_train)
            else:
                raise e  # 若是其他錯誤就直接拋出

        y_pred = model_fold.predict(X_test)
        y_prob = model_fold.predict_proba(X_test)[:, 1]

        # ROC
        fpr, tpr, _ = roc_curve(y_test, y_prob)
        roc_auc = auc(fpr, tpr)
        tpr_interp = np.interp(mean_fpr, fpr, tpr)
        tpr_interp[0] = 0.0
        tprs.append(tpr_interp)
        aucs.append(roc_auc)

        # Metrics
        accs.append(accuracy_score(y_test, y_pred))
        precs.append(precision_score(y_test, y_pred))
        recalls.append(recall_score(y_test, y_pred))
        f1s.append(f1_score(y_test, y_pred))
        all_cm += confusion_matrix(y_test, y_pred)

    trained_models[model_name] = copy.deepcopy(model_fold)

    # ROC Curve
    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)

    fig_roc.add_trace(go.Scatter(
        x=mean_fpr, y=mean_tpr, mode='lines',
        name=f"{model_name} (mean AUC={mean_auc:.3f})"
    ))

    results.append({
        'Model': model_name,
        'Accuracy Mean': np.mean(accs),
        'Accuracy Std': np.std(accs),
        'Precision Mean': np.mean(precs),
        'Recall Mean': np.mean(recalls),
        'F1 Score Mean': np.mean(f1s),
        'ROC AUC Mean': np.mean(aucs),
        'ROC AUC Std': np.std(aucs),
        'Confusion Matrix': all_cm
    })

# Add Random Baseline
fig_roc.add_trace(go.Scatter(
    x=[0, 1], y=[0, 1], mode='lines',
    line=dict(dash='dash'), name='Random Baseline'
))

fig_roc.update_layout(
    title="ROC Curve Comparison (Cross-Validation Mean)",
    xaxis_title="False Positive Rate",
    yaxis_title="True Positive Rate",
    width=800, height=600
)

fig_roc.show()

▶ Running CV for: HybridXGBRF (Our Approach)
▶ Running CV for: LogisticRegression (max_iter=200)


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

/usr/local/lib/python3.12/dist-packages/sklearn/

▶ Running CV for: XGBClassifier
▶ Running CV for: RandomForestClassifier
▶ Running CV for: LogisticRegression (max_iter=1000)
▶ Running CV for: Ridge


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge



▶ Running CV for: Lasso


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge



▶ Running CV for: Elastic


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge



In [ ]:
import shap

# 訓練模型（完整資料）
xgb_model = all_models["HybridXGBRF (Our Approach)"]
xgb_model.fit(X, y)

# 建立 SHAP 解釋器
explainer = shap.TreeExplainer(xgb_model)
shap_values = explainer.shap_values(X)

# 計算平均 SHAP 值絕對值（作為重要性）
import numpy as np
shap_abs_mean = np.abs(shap_values).mean(axis=0)

In [ ]:
import pandas as pd
import plotly.express as px

# 整理成 DataFrame 並排序
importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Mean |SHAP Value|': shap_abs_mean
}).sort_values(by='Mean |SHAP Value|', ascending=False)

# 畫前 20 名
top_n = 20
fig_bar = px.bar(
    importance_df.head(top_n),
    x='Mean |SHAP Value|',
    y='Feature',
    orientation='h',
    title="🎯 Top SHAP Features by Mean |SHAP|",
)
fig_bar.update_layout(yaxis=dict(categoryorder='total ascending'))
fig_bar.show()

In [ ]:
import plotly.graph_objects as go

for r in results:
    cm = r['Confusion Matrix']
    model_name = r['Model']

    z = cm
    x_labels = ['Predicted 0', 'Predicted 1']
    y_labels = ['Actual 0', 'Actual 1']

    fig_cm = go.Figure(data=go.Heatmap(
        z=z,
        x=x_labels,
        y=y_labels,
        colorscale='Blues',
        text=z,
        texttemplate="%{text}"
    ))

    fig_cm.update_layout(
        title=f"Confusion Matrix - {model_name}",
        xaxis_title="Predicted Label",
        yaxis_title="True Label",
        width=500,
        height=500
    )

    fig_cm.show()

In [ ]:
# Convert results to DataFrame
results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by="ROC AUC Mean", ascending=False).reset_index(drop=True)
results_df

,Model,Accuracy Mean,Accuracy Std,Precision Mean,Recall Mean,F1 Score Mean,ROC AUC Mean,ROC AUC Std,Confusion Matrix
0,HybridXGBRF (Our Approach),0.814735,0.000449,0.639981,0.114757,0.194465,0.822228,0.004230,"[[21423, 342], [4667, 605]]"
1,XGBClassifier,0.816844,0.002195,0.632419,0.146628,0.237610,0.821518,0.003884,"[[21312, 453], [4499, 773]]"
2,LogisticRegression (max_iter=1000),0.808596,0.001596,0.570868,0.071892,0.127564,0.790071,0.004654,"[[21483, 282], [4893, 379]]"
3,LogisticRegression (max_iter=200),0.807523,0.001525,0.555511,0.066391,0.118167,0.789246,0.004149,"[[21483, 282], [4922, 350]]"
4,Lasso,0.808448,0.001960,0.568825,0.070375,0.125120,0.788786,0.004692,"[[21487, 278], [4901, 371]]"
5,Elastic,0.808263,0.001964,0.565634,0.068858,0.122605,0.787750,0.004693,"[[21490, 275], [4909, 363]]"
6,Ridge,0.808078,0.001995,0.562952,0.067340,0.120153,0.786791,0.004705,"[[21493, 272], [4917, 355]]"
7,RandomForestClassifier,0.793801,0.002634,0.460576,0.335360,0.388035,0.770692,0.003684,"[[19694, 2071], [3504, 1768]]"


# 測試外部資料在 XGBoost 模型下的結果

In [ ]:
external.head()

,H01_NUM,dbname,入家日期,結案日期,死亡標記,觀察天數,性別_is_male,預估年齡,DNR_flag,ADL_總分_max,...,first_ 意識總分,last_ 意識總分,意識總分Max,意識總分_diff,意識分級,使用呼吸輔具,first_has_feeding_tube,last_has_feeding_tube,diff_has_feeding_tube,had_fall
0,1376,NaN,NaN,NaN,0,197,1,77,0,90,...,15.0,15.0,15.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0
1,1322,NaN,NaN,NaN,0,327,1,92,0,10,...,12.0,12.0,12.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0
2,1319,NaN,NaN,NaN,0,255,1,78,1,5,...,11.0,10.0,11.0,-1.0,NaN,0.0,0.0,0.0,0.0,0.0
3,1333,NaN,NaN,NaN,0,293,1,82,1,10,...,12.0,12.0,12.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0
4,1452,NaN,NaN,NaN,0,341,0,80,1,10,...,11.0,11.0,11.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0


In [ ]:
ex_X = external[features].drop(columns=['死亡標記'])
ex_y = external['死亡標記']

In [ ]:
ex_X = ex_X.fillna(0)

In [ ]:
# 丟掉這些欄位
ex_X = ex_X.drop(columns=drop_columns)

In [ ]:
ex_X.describe().T

,count,mean,std,min,25%,50%,75%,max
性別_is_male,6339.0,0.508440,0.499968,0.0,0.0,1.0,1.000000,1.000000
預估年齡,6339.0,78.607825,11.742108,1.0,72.0,81.0,87.000000,124.000000
ADL_Max,6339.0,29.695536,31.687105,0.0,0.0,20.0,50.000000,100.000000
ADL_總分_max,6339.0,29.695536,31.687105,0.0,0.0,20.0,50.000000,100.000000
DNR_flag,6339.0,0.456854,0.498174,0.0,0.0,0.0,1.000000,1.000000
ADL_Min,6339.0,21.894621,28.481462,0.0,0.0,10.0,35.000000,100.000000
ADL_last_CouldNot,6339.0,0.010569,0.102271,0.0,0.0,0.0,0.000000,1.000000
ADL_first_CouldNot,6339.0,0.010412,0.101513,0.0,0.0,0.0,0.000000,1.000000
ADL_明顯惡化,6339.0,0.127149,0.333167,0.0,0.0,0.0,0.000000,1.000000
六個月內住院次數,6339.0,2.056791,9.258256,0.0,0.0,1.0,2.000000,581.000000


In [ ]:
eX_missing_info = ex_X.isnull().sum().to_frame(name='Missing Count')
eX_missing_info['Missing Ratio'] = (eX_missing_info['Missing Count'] / len(ex_X)).round(4)
eX_missing_info = eX_missing_info.sort_values(by='Missing Ratio', ascending=False)
eX_missing_info

,Missing Count,Missing Ratio
性別_is_male,0,0.0
預估年齡,0,0.0
ADL_Max,0,0.0
ADL_總分_max,0,0.0
DNR_flag,0,0.0
ADL_Min,0,0.0
ADL_last_CouldNot,0,0.0
ADL_first_CouldNot,0,0.0
ADL_明顯惡化,0,0.0
六個月內住院次數,0,0.0


In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.base import clone
from sklearn.exceptions import NotFittedError
from sklearn.utils.validation import check_is_fitted
import plotly.subplots as sp

def evaluate_all_models_visual(models: dict, X_val, y_val):
    mean_fpr = np.linspace(0, 1, 100)
    fig_roc = go.Figure()
    results = []

    # 建立混淆矩陣子圖
    num_models = len(models)
    cols = 3
    rows = int(np.ceil(num_models / cols))

    fig_cm = sp.make_subplots(
        rows=rows, cols=cols,
        subplot_titles=list(models.keys()),
        horizontal_spacing=0.15,
        vertical_spacing=0.15
    )

    for i, (model_name, model) in enumerate(models.items()):
        print(f"🔍 Evaluating {model_name}...")

        # 嘗試使用原始資料
        X_input = X_val.copy()
        y_input = y_val

        # 若模型不支援 NaN，則補值
        try:
            # 嘗試呼叫 predict_proba
            _ = model.predict_proba(X_input)
        except ValueError as e:
            if "Input X contains NaN" in str(e):
                print(f"⚠️  {model_name} 不支援 NaN，自動補值中...")
                imputer = SimpleImputer(strategy="median")
                X_input = pd.DataFrame(imputer.fit_transform(X_input), columns=X_val.columns)
            else:
                raise e

        # 預測
        y_pred = model.predict(X_input)
        y_prob = model.predict_proba(X_input)[:, 1]

        # 指標
        acc = accuracy_score(y_input, y_pred)
        prec = precision_score(y_input, y_pred)
        rec = recall_score(y_input, y_pred)
        f1 = f1_score(y_input, y_pred)
        auc_val = roc_auc_score(y_input, y_prob)

        # ROC
        fpr, tpr, _ = roc_curve(y_input, y_prob)
        tpr_interp = np.interp(mean_fpr, fpr, tpr)
        tpr_interp[0], tpr_interp[-1] = 0.0, 1.0

        fig_roc.add_trace(go.Scatter(
            x=mean_fpr, y=tpr_interp,
            mode='lines',
            name=f"{model_name} (AUC={auc_val:.3f})"
        ))

        # 混淆矩陣
        cm = confusion_matrix(y_input, y_pred)
        row, col = i // cols + 1, i % cols + 1
        fig_cm.add_trace(
            go.Heatmap(
                z=cm,
                x=["Predicted Negative", "Predicted Positive"],
                y=["Actual Negative", "Actual Positive"],
                colorscale='Blues',
                text=cm,
                texttemplate="%{text}",
                showscale=False
            ),
            row=row, col=col
        )

        results.append({
            "Model": model_name,
            "Accuracy": acc,
            "Precision": prec,
            "Recall": rec,
            "F1 Score": f1,
            "ROC AUC": auc_val
        })

    # 隨機基準線
    fig_roc.add_trace(go.Scatter(
        x=[0, 1], y=[0, 1],
        mode='lines',
        line=dict(dash='dash'),
        name='Random Baseline'
    ))

    fig_roc.update_layout(
        title="ROC Curve Comparison",
        xaxis_title="False Positive Rate",
        yaxis_title="True Positive Rate",
        width=800,
        height=600
    )
    fig_roc.show()

    fig_cm.update_layout(
        title="Confusion Matrices of All Models",
        width=400 * cols,
        height=300 * rows,
        showlegend=False
    )
    fig_cm.show()

    # 指標表格
    df_result = pd.DataFrame(results)

    return df_result

In [ ]:
# 假設已經訓練完模型並存在 trained_models 中
evaluate_all_models_visual(trained_models, ex_X, ex_y)

🔍 Evaluating HybridXGBRF (Our Approach)...
🔍 Evaluating LogisticRegression (max_iter=200)...
🔍 Evaluating XGBClassifier...
🔍 Evaluating RandomForestClassifier...
🔍 Evaluating LogisticRegression (max_iter=1000)...
🔍 Evaluating Ridge...
🔍 Evaluating Lasso...
🔍 Evaluating Elastic...


,Model,Accuracy,Precision,Recall,F1 Score,ROC AUC
0,HybridXGBRF (Our Approach),0.738918,0.742308,0.108366,0.189123,0.796645
1,LogisticRegression (max_iter=200),0.729610,0.710692,0.063448,0.116495,0.757667
2,XGBClassifier,0.747121,0.727041,0.160022,0.262310,0.796819
3,RandomForestClassifier,0.737025,0.550802,0.346996,0.425766,0.745888
4,LogisticRegression (max_iter=1000),0.734027,0.705628,0.091522,0.162028,0.759670
5,Ridge,0.733554,0.709091,0.087591,0.155922,0.758115
6,Lasso,0.733396,0.700441,0.089276,0.158367,0.759051
7,Elastic,0.733396,0.702222,0.088714,0.157527,0.758570


In [ ]:
# ===== 次族群實驗：年齡、ADL 變化、性別 =====
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

def _first_existing_column(df, candidates):
    for c in candidates:
        if c in df.columns:
            return c
    return None

def _make_sex_masks(df):
    """
    回傳 {'男性': mask, '女性': mask}；若無法判斷，回傳空 dict。
    支援：
      1) '性別' 欄位（值可能為 '男'/'女' 或 1/0/2…）
      2) one-hot 欄位 '性別_男' / '性別_女'
    """
    masks = {}
    if '性別_is_male' in df.columns:
        col = df['性別_is_male']
        # 嘗試各種常見標記
        male_mask = col.astype(str).str.contains('男') | (col == 1) | (col.astype(str).str.lower().isin(['m','male']))
        female_mask = col.astype(str).str.contains('女') | (col == 0) | (col.astype(str).str.lower().isin(['f','female']))
        if male_mask.any(): masks['男性'] = male_mask
        if female_mask.any(): masks['女性'] = female_mask
    else:
        male_col = _first_existing_column(df, ['性別_男','男','male','Male','M'])
        female_col = _first_existing_column(df, ['性別_女','女','female','Female','F'])
        if male_col is not None:
            masks['男性'] = df[male_col] == 1
        if female_col is not None:
            masks['女性'] = df[female_col] == 1
    return masks

def _make_adl_change_masks(df):
    """
    建立 ADL 變好/變差遮罩：
    'ADL_明顯惡化'（=0 視為變好，=1 變差）
    """
    masks = {}
    masks['ADL 變好'] = df['ADL_明顯惡化'] == 0
    masks['ADL 變差'] = df['ADL_明顯惡化'] == 1
    return masks

def _compute_metrics(y_true, y_prob, y_pred):
    return {
        'Accuracy': accuracy_score(y_true, y_pred),
        'Precision': precision_score(y_true, y_pred, zero_division=0),
        'Recall': recall_score(y_true, y_pred, zero_division=0),
        'F1': f1_score(y_true, y_pred, zero_division=0),
        'ROC AUC': roc_auc_score(y_true, y_prob) if len(np.unique(y_true)) > 1 else np.nan,
        'Support (n)': int(len(y_true)),
        'Positives (n)': int(y_true.sum())
    }

def evaluate_subgroups(models: dict, X_all: pd.DataFrame, y_all: pd.Series, raw_df_for_masks: pd.DataFrame):
    """
    models: 已訓練好的模型字典 trained_models
    X_all, y_all: 用於評估的特徵與標記（例如 ex_X, ex_y）
    raw_df_for_masks: 與 X_all 對齊、包含「年齡/ADL/性別」原始欄位的 DataFrame（例如 external）
    """
    # 年齡遮罩（需 '預估年齡'）
    subgroup_masks = {}
    if '預估年齡' in raw_df_for_masks.columns:
        subgroup_masks['年齡 > 85'] = raw_df_for_masks['預估年齡'] > 85
        subgroup_masks['年齡 <= 85'] = raw_df_for_masks['預估年齡'] <= 85
    else:
        print("⚠️ 找不到欄位『預估年齡』，跳過年齡分組。")

    # ADL 變化遮罩
    adl_masks = _make_adl_change_masks(raw_df_for_masks)
    if adl_masks:
        subgroup_masks.update(adl_masks)
    else:
        print("⚠️ 找不到可推算 ADL 變化的欄位，跳過 ADL 分組。")

    # 性別遮罩
    sex_masks = _make_sex_masks(raw_df_for_masks)
    if sex_masks:
        subgroup_masks.update(sex_masks)
    else:
        print("⚠️ 找不到可用的性別欄位，跳過性別分組。")

    rows = []
    for subgroup_name, mask in subgroup_masks.items():
        mask = mask.fillna(False).astype(bool)  # 安全轉型
        if mask.sum() == 0:
            print(f"⚠️ 次族群「{subgroup_name}」資料筆數為 0，略過。")
            continue

        X_sub = X_all.loc[mask]
        y_sub = y_all.loc[mask]

        # 若模型不支援 NaN，與你上面一致，統一補值策略（中位數）
        from sklearn.impute import SimpleImputer
        imputer = SimpleImputer(strategy="median")
        X_sub_imp = pd.DataFrame(imputer.fit_transform(X_sub), columns=X_sub.columns, index=X_sub.index)

        for model_name, model in models.items():
            # 預測
            y_prob = model.predict_proba(X_sub_imp)[:, 1]
            y_pred = (y_prob >= 0.5).astype(int)

            metrics = _compute_metrics(y_sub, y_prob, y_pred)
            metrics.update({'Subgroup': subgroup_name, 'Model': model_name})
            rows.append(metrics)

    result_df = pd.DataFrame(rows)
    if not result_df.empty:
        # 排序：先按次族群，再按 ROC AUC 由高到低
        result_df = result_df.sort_values(by=['Subgroup','ROC AUC'], ascending=[True, False]).reset_index(drop=True)
    return result_df

# === 執行：以外部驗證集為例 ===
subgroup_results = evaluate_subgroups(trained_models, ex_X, ex_y, external)
display(subgroup_results)

,Accuracy,Precision,Recall,F1,ROC AUC,Support (n),Positives (n),Subgroup,Model
0,0.732333,0.727041,0.171790,0.277913,0.800108,5533,1659,ADL 變好,XGBClassifier
1,0.722935,0.742308,0.116335,0.201146,0.798895,5533,1659,ADL 變好,HybridXGBRF (Our Approach)
2,0.715344,0.716495,0.083785,0.150027,0.750555,5533,1659,ADL 變好,LogisticRegression (max_iter=1000)
3,0.714441,0.704663,0.081977,0.146868,0.749909,5533,1659,ADL 變好,Lasso
4,0.714079,0.699482,0.081374,0.145788,0.749428,5533,1659,ADL 變好,Elastic
5,0.714260,0.703125,0.081374,0.145867,0.748997,5533,1659,ADL 變好,Ridge
6,0.710103,0.727273,0.053044,0.098876,0.748093,5533,1659,ADL 變好,LogisticRegression (max_iter=200)
7,0.719682,0.547957,0.371911,0.443088,0.739875,5533,1659,ADL 變好,RandomForestClassifier
8,0.862283,0.648649,0.196721,0.301887,0.701694,806,122,ADL 變差,LogisticRegression (max_iter=1000)
9,0.863524,0.676471,0.188525,0.294872,0.699358,806,122,ADL 變差,Lasso


In [ ]:
# 若只想看主力模型
display(subgroup_results[subgroup_results['Model'] == 'HybridXGBRF (Our Approach)'])

,Accuracy,Precision,Recall,F1,ROC AUC,Support (n),Positives (n),Subgroup,Model
1,0.722935,0.742308,0.116335,0.201146,0.798895,5533,1659,ADL 變好,HybridXGBRF (Our Approach)
14,0.848635,0.000000,0.000000,0.000000,0.684486,806,122,ADL 變差,HybridXGBRF (Our Approach)
16,0.770860,0.000000,0.000000,0.000000,0.787993,3116,714,女性,HybridXGBRF (Our Approach)
25,0.744000,0.747967,0.077900,0.141104,0.794274,4375,1181,年齡 <= 85,HybridXGBRF (Our Approach)
32,0.727597,0.737226,0.168333,0.274084,0.800214,1964,600,年齡 > 85,HybridXGBRF (Our Approach)
41,0.708036,0.742308,0.180881,0.290882,0.796041,3223,1067,男性,HybridXGBRF (Our Approach)


In [ ]:
# ===== 複合次族群：年齡 × 性別 × ADL是否明顯惡化 =====
from itertools import product
import pandas as pd
from sklearn.impute import SimpleImputer

def evaluate_age_sex_composites(
    models: dict,
    X_all: pd.DataFrame,
    y_all: pd.Series,
    raw_df_for_masks: pd.DataFrame,
    age_threshold: int = 85,
    min_support: int = 10   # 次族群最少樣本數，太小就略過避免不穩定
):
    """
    針對「年齡(>threshold / <=threshold) × 性別(男性/女性) × ADL(變好/變差)」的交叉次族群做評估。
    會輸出每個模型在各交叉次族群的 Accuracy / Precision / Recall / F1 / ROC-AUC 等。
    依賴你已定義的: _make_sex_masks, _make_adl_change_masks, _compute_metrics。
    """
    if '預估年齡' not in raw_df_for_masks.columns:
        print("⚠️ 找不到欄位『預估年齡』，無法建立年齡遮罩。")
        return pd.DataFrame()

    # 年齡 bins
    age_masks = {
        f'年齡 > {age_threshold}': (raw_df_for_masks['預估年齡'] > age_threshold),
        f'年齡 <= {age_threshold}': (raw_df_for_masks['預估年齡'] <= age_threshold),
    }

    # 性別 masks（沿用你上面的 _make_sex_masks）
    sex_masks = _make_sex_masks(raw_df_for_masks)
    if not sex_masks:
        print("⚠️ 找不到可用的性別欄位，無法建立性別遮罩。")
        return pd.DataFrame()

    # ADL 變化 masks（沿用你上面的 _make_adl_change_masks）
    if 'ADL_明顯惡化' not in raw_df_for_masks.columns:
        print("⚠️ 找不到欄位『ADL_明顯惡化』，無法建立 ADL 遮罩。")
        return pd.DataFrame()
    adl_masks = _make_adl_change_masks(raw_df_for_masks)  # {'ADL 變好': mask, 'ADL 變差': mask}

    rows = []
    for (age_name, age_mask), (sex_name, sex_mask), (adl_name, adl_mask) in product(
        age_masks.items(), sex_masks.items(), adl_masks.items()
    ):
        combo_name = f"{age_name} & {sex_name} & {adl_name}"
        mask = (
            age_mask.fillna(False).astype(bool)
            & sex_mask.fillna(False).astype(bool)
            & adl_mask.fillna(False).astype(bool)
        )
        n = int(mask.sum())
        if n < min_support:
            print(f"ℹ️ 複合次族群「{combo_name}」樣本數 {n} < min_support={min_support}，略過。")
            continue

        X_sub = X_all.loc[mask]
        y_sub = y_all.loc[mask]

        # 與你現有策略一致：補缺失值（中位數）
        imputer = SimpleImputer(strategy="median")
        X_sub_imp = pd.DataFrame(imputer.fit_transform(X_sub), columns=X_sub.columns, index=X_sub.index)

        for model_name, model in models.items():
            y_prob = model.predict_proba(X_sub_imp)[:, 1]
            y_pred = (y_prob >= 0.5).astype(int)

            metrics = _compute_metrics(y_sub, y_prob, y_pred)
            metrics.update({'Subgroup': combo_name, 'Model': model_name})
            rows.append(metrics)

    result_df = pd.DataFrame(rows)
    if not result_df.empty:
        result_df['Prevalence'] = result_df['Positives (n)'] / result_df['Support (n)']
        result_df = result_df.sort_values(by=['Subgroup', 'ROC AUC'], ascending=[True, False]).reset_index(drop=True)
    return result_df

In [ ]:
age_sex_results = evaluate_age_sex_composites(
    trained_models, ex_X, ex_y, external,
    age_threshold=85,
    min_support=10
)
display(age_sex_results)

pivot_auc = age_sex_results.pivot_table(index='Subgroup', columns='Model', values='ROC AUC')
display(pivot_auc)

,Accuracy,Precision,Recall,F1,ROC AUC,Support (n),Positives (n),Subgroup,Model,Prevalence
0,0.781630,1.000000,0.002778,0.005540,0.797910,1644,360,年齡 <= 85 & 女性 & ADL 變好,XGBClassifier,0.218978
1,0.781022,0.000000,0.000000,0.000000,0.797823,1644,360,年齡 <= 85 & 女性 & ADL 變好,HybridXGBRF (Our Approach),0.218978
2,0.779197,0.000000,0.000000,0.000000,0.739254,1644,360,年齡 <= 85 & 女性 & ADL 變好,LogisticRegression (max_iter=200),0.218978
3,0.781022,0.500000,0.002778,0.005525,0.737680,1644,360,年齡 <= 85 & 女性 & ADL 變好,LogisticRegression (max_iter=1000),0.218978
4,0.780414,0.333333,0.002778,0.005510,0.736795,1644,360,年齡 <= 85 & 女性 & ADL 變好,Lasso,0.218978
...,...,...,...,...,...,...,...,...,...,...
59,0.864865,0.875000,0.333333,0.482759,0.697354,111,21,年齡 > 85 & 男性 & ADL 變差,Lasso,0.189189
60,0.864865,0.875000,0.333333,0.482759,0.695767,111,21,年齡 > 85 & 男性 & ADL 變差,Elastic,0.189189
61,0.855856,0.857143,0.285714,0.428571,0.692593,111,21,年齡 > 85 & 男性 & ADL 變差,Ridge,0.189189
62,0.810811,0.000000,0.000000,0.000000,0.639418,111,21,年齡 > 85 & 男性 & ADL 變差,HybridXGBRF (Our Approach),0.189189


Model,Elastic,HybridXGBRF (Our Approach),Lasso,LogisticRegression (max_iter=1000),LogisticRegression (max_iter=200),RandomForestClassifier,Ridge,XGBClassifier
Subgroup,,,,,,,,
年齡 <= 85 & 女性 & ADL 變好,0.736256,0.797823,0.736795,0.737680,0.739254,0.728950,0.735854,0.797910
年齡 <= 85 & 女性 & ADL 變差,0.651082,0.689466,0.652814,0.655844,0.656999,0.722006,0.648629,0.638961
年齡 <= 85 & 男性 & ADL 變好,0.739595,0.790669,0.740248,0.741127,0.732417,0.739663,0.739038,0.793747
年齡 <= 85 & 男性 & ADL 變差,0.738143,0.698844,0.739149,0.741830,0.732278,0.665452,0.737305,0.653888
年齡 > 85 & 女性 & ADL 變好,0.716417,0.773573,0.716700,0.716830,0.719312,0.700296,0.715962,0.774947
年齡 > 85 & 女性 & ADL 變差,0.649804,0.662353,0.652157,0.653333,0.641961,0.604314,0.644706,0.608235
年齡 > 85 & 男性 & ADL 變好,0.755862,0.808214,0.755893,0.756952,0.760204,0.745134,0.755480,0.802453
年齡 > 85 & 男性 & ADL 變差,0.695767,0.639418,0.697354,0.699471,0.699471,0.729365,0.692593,0.629630


In [ ]:
# 每個模型在哪個次族群表現最好（同樣以 ROC AUC 為主）
def best_subgroup_per_model(results_df: pd.DataFrame,
                            primary='ROC AUC',
                            tie_breakers=('F1','Recall','Precision','Accuracy','Support (n)')):
    sort_cols = ['Model', primary, *tie_breakers]
    sort_asc  = [True, False, *([False]*len(tie_breakers))]
    df_sorted = results_df.sort_values(by=sort_cols, ascending=sort_asc)
    best_df = df_sorted.groupby('Model', as_index=False).head(1).reset_index(drop=True)
    return best_df

best_subgroup_each_model = best_subgroup_per_model(age_sex_results)
display(best_subgroup_each_model)

,Accuracy,Precision,Recall,F1,ROC AUC,Support (n),Positives (n),Subgroup,Model,Prevalence
0,0.644231,0.743243,0.213178,0.331325,0.755862,624,258,年齡 > 85 & 男性 & ADL 變好,Elastic,0.413462
1,0.690705,0.737226,0.391473,0.511392,0.808214,624,258,年齡 > 85 & 男性 & ADL 變好,HybridXGBRF (Our Approach),0.413462
2,0.644231,0.743243,0.213178,0.331325,0.755893,624,258,年齡 > 85 & 男性 & ADL 變好,Lasso,0.413462
3,0.644231,0.743243,0.213178,0.331325,0.756952,624,258,年齡 > 85 & 男性 & ADL 變好,LogisticRegression (max_iter=1000),0.413462
4,0.625000,0.875000,0.108527,0.193103,0.760204,624,258,年齡 > 85 & 男性 & ADL 變好,LogisticRegression (max_iter=200),0.413462
5,0.698718,0.675000,0.523256,0.589520,0.745134,624,258,年齡 > 85 & 男性 & ADL 變好,RandomForestClassifier,0.413462
6,0.645833,0.753425,0.213178,0.332326,0.755480,624,258,年齡 > 85 & 男性 & ADL 變好,Ridge,0.413462
7,0.693910,0.731034,0.410853,0.526055,0.802453,624,258,年齡 > 85 & 男性 & ADL 變好,XGBClassifier,0.413462


In [ ]:
# 直接列出「整體表現最高的 (Subgroup, Model) Top-K」
def top_k_overall(results_df: pd.DataFrame, k=10,
                  primary='ROC AUC',
                  tie_breakers=('F1','Recall','Precision','Accuracy','Support (n)')):
    sort_cols = [primary, *tie_breakers]
    sort_asc  = [False, *([False]*len(tie_breakers))]
    return results_df.sort_values(by=sort_cols, ascending=sort_asc).head(k).reset_index(drop=True)

top10 = top_k_overall(age_sex_results, k=10)
display(top10)

,Accuracy,Precision,Recall,F1,ROC AUC,Support (n),Positives (n),Subgroup,Model,Prevalence
0,0.690705,0.737226,0.391473,0.511392,0.808214,624,258,年齡 > 85 & 男性 & ADL 變好,HybridXGBRF (Our Approach),0.413462
1,0.693910,0.731034,0.410853,0.526055,0.802453,624,258,年齡 > 85 & 男性 & ADL 變好,XGBClassifier,0.413462
2,0.781630,1.000000,0.002778,0.005540,0.797910,1644,360,年齡 <= 85 & 女性 & ADL 變好,XGBClassifier,0.218978
3,0.781022,0.000000,0.000000,0.000000,0.797823,1644,360,年齡 <= 85 & 女性 & ADL 變好,HybridXGBRF (Our Approach),0.218978
4,0.715388,0.723577,0.241520,0.362157,0.793747,2203,737,年齡 <= 85 & 男性 & ADL 變好,XGBClassifier,0.334544
5,0.693146,0.747967,0.124830,0.213953,0.790669,2203,737,年齡 <= 85 & 男性 & ADL 變好,HybridXGBRF (Our Approach),0.334544
6,0.713748,0.000000,0.000000,0.000000,0.774947,1062,304,年齡 > 85 & 女性 & ADL 變好,XGBClassifier,0.286252
7,0.713748,0.000000,0.000000,0.000000,0.773573,1062,304,年齡 > 85 & 女性 & ADL 變好,HybridXGBRF (Our Approach),0.286252
8,0.625000,0.875000,0.108527,0.193103,0.760204,624,258,年齡 > 85 & 男性 & ADL 變好,LogisticRegression (max_iter=200),0.413462
9,0.644231,0.743243,0.213178,0.331325,0.756952,624,258,年齡 > 85 & 男性 & ADL 變好,LogisticRegression (max_iter=1000),0.413462


In [ ]:
# 每個次族群下，表現最好的模型（以 ROC AUC 為主，F1/Recall/Precision/Accuracy/Support 作為平手時的次序）
def best_model_per_subgroup(results_df: pd.DataFrame,
                            primary='ROC AUC',
                            tie_breakers=('F1','Recall','Precision','Accuracy','Support (n)')):
    sort_cols = ['Subgroup', primary, *tie_breakers]
    sort_asc  = [True, False, *([False]*len(tie_breakers))]
    df_sorted = results_df.sort_values(by=sort_cols, ascending=sort_asc)
    # 取每個 Subgroup 的第一列（即最佳模型）
    best_df = df_sorted.groupby('Subgroup', as_index=False).head(1).reset_index(drop=True)
    return best_df

best_by_subgroup = best_model_per_subgroup(age_sex_results)
display(best_by_subgroup)

,Accuracy,Precision,Recall,F1,ROC AUC,Support (n),Positives (n),Subgroup,Model,Prevalence
0,0.781630,1.000000,0.002778,0.005540,0.797910,1644,360,年齡 <= 85 & 女性 & ADL 變好,XGBClassifier,0.218978
1,0.868313,0.666667,0.060606,0.111111,0.722006,243,33,年齡 <= 85 & 女性 & ADL 變差,RandomForestClassifier,0.135802
2,0.715388,0.723577,0.241520,0.362157,0.793747,2203,737,年齡 <= 85 & 男性 & ADL 變好,XGBClassifier,0.334544
3,0.838596,0.647059,0.215686,0.323529,0.741830,285,51,年齡 <= 85 & 男性 & ADL 變差,LogisticRegression (max_iter=1000),0.178947
4,0.713748,0.000000,0.000000,0.000000,0.774947,1062,304,年齡 > 85 & 女性 & ADL 變好,XGBClassifier,0.286252
5,0.898204,0.000000,0.000000,0.000000,0.662353,167,17,年齡 > 85 & 女性 & ADL 變差,HybridXGBRF (Our Approach),0.101796
6,0.690705,0.737226,0.391473,0.511392,0.808214,624,258,年齡 > 85 & 男性 & ADL 變好,HybridXGBRF (Our Approach),0.413462
7,0.819820,1.000000,0.047619,0.090909,0.729365,111,21,年齡 > 85 & 男性 & ADL 變差,RandomForestClassifier,0.189189
